# COVID-19 in Italy
* Data analysis and visualization for COVID-19 cases in Italy.
* Data available at https://github.com/pcm-dpc/COVID-19 (updates every day at 17.00 - 18.00)

In [ ]:
# data manipulation
import numpy as np
import pandas as pd

# plotting libraries
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

# color palette
cnf = '#475586' # confirmed
dth = '#c54b46' # death 
rec = '#edba62' # recovered
act = '#22728f' # active case 
osp = '#ff9135' # hospitalized
tint = '#ee4242' # intensive care  


### National trends 
> Load data file, rename and polish columns

In [ ]:
data=pd.read_csv("../input/covid-19-italy-updated-regularly/national_data.csv")

data["date"] = pd.to_datetime(data["date"], errors='coerce').dt.date

## Total cases breakdown

In [ ]:
temp = data.copy()
temp = temp.groupby('date')['total_positive_cases', 'death', 'recovered', 'total_currently_positive'].sum().reset_index()
temp = temp[temp['date']==max(temp['date'])].reset_index(drop=True)
temp

In [ ]:
temp = data.copy()
temp = temp.groupby('date')['total_positive_cases', 'death', 'recovered', 'total_currently_positive'].sum().reset_index()

fig = go.Figure(data=[
    go.Bar(name='Actual cases', x=temp["date"], y=temp["total_currently_positive"], marker=go.bar.Marker(
    color=act)),
    go.Bar(name='Recovered', x=temp["date"], y=temp["recovered"],marker=go.bar.Marker(
    color=rec)),
    go.Bar(name='Deceased', x=temp["date"], y=temp["death"],marker=go.bar.Marker(
    color=dth))
])
# Change the bar mode
fig.update_layout(
    width = 700,
    height=500,
    barmode='stack', 
    title='Total cases',
    font=dict(size=14, color='#686868'),
    xaxis_tickangle=-45,
    yaxis=dict(
        title='Count'
        ))

fig.show()

**Hospitalized situation **

> Tracking the numbers of hospitalized people and in particular of insive care patients is pf particular importance for Italy at the moment, due to the (relative) low number of intensive care beds nationwide.

In [ ]:
temp = data.groupby('date')['hospitalized_with_symptoms', 'intensive_care'].sum().reset_index()


fig = go.Figure(data=[
    go.Bar(name='Hospitalized', x=temp["date"], y=temp["hospitalized_with_symptoms"],marker=go.bar.Marker(
    color=osp)),
    go.Bar(name='Intensive Care', x=temp["date"], y=temp["intensive_care"],marker=go.bar.Marker(
    color=tint))
])
# Change the bar mode
fig.update_layout(
    width = 700,
    height = 500,
    barmode='stack', 
    title='Total Hospitalized',
    font=dict(size=14, color='#686868'),
    xaxis_tickangle=-45,
    yaxis=dict(
        title='Count'
        ))

fig.show()

**Daily actual cases increment**

> Please note that 'New actual cases' = new positive tests - deaths that day - recovered that day

In [ ]:
temp = data.groupby('date')['new_currently_positive', 'total_tests'].sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=temp['total_tests'], y=temp['new_currently_positive'],
    mode='lines+markers',
    xaxis='x1'
))


fig.add_trace(go.Scatter(
    x=temp['date'], y=temp['new_currently_positive'],
    mode='lines+markers',
    xaxis='x2',
    marker=dict(
        color='#ea4335'
    )
))

fig.update_layout(
    width=600,
    height=500,
    font=dict(size=14, color='#686868'),
    xaxis=dict(
        title='Tests performed'
    ),
    xaxis2=dict(
        side='top',
        showgrid=False
    ),
    xaxis2_tickangle=-45,
    yaxis=dict(
        title='New actual cases'
    ),
    showlegend=False,
    annotations=[
        dict(
            x=76000,
            y=450,
            xref="x",
            yref="y",
            text="Incorrect tests",
            showarrow=True,
            arrowhead=2,
            arrowcolor="#636363",
            ax=-20,
            ay=40
        )
    ]
)

fig.show()

**Mortality **

In [ ]:
dth_ita = data.groupby('date')['death'].sum().reset_index()

fig = go.Figure(go.Scatter(
    x=dth_ita['date'], y=dth_ita['death'],
    mode='lines+markers',
    marker=dict(
        color='#4285f4'
    )
))

fig.update_layout(
    width=600,
    height=500,
    font=dict(size=14, color='#686868'),
    xaxis_tickangle=-45,
    yaxis=dict(
        title='Deceased'
        ),
    yaxis_type='log'
)
fig.show()

## Cases per region

In [ ]:
data= pd.read_csv('../input/covid-19-italy-updated-regularly/regional_data.csv')

data["date"] = pd.to_datetime(data['date'], errors='coerce').dt.date
data_latest = data[data['date']==data['date'].max()]

In [ ]:
fig = px.bar(data_latest.sort_values('total_positive_cases', ascending=False).head(20).sort_values('total_positive_cases', ascending=True), 
             x="total_positive_cases", y="region_denomination", title='Cases per region', text='total_positive_cases', orientation='h', 
             width=600, height=600, range_x = [0, 4.7])
fig.update_traces(marker_color=cnf, opacity=0.6, textposition='outside')
fig.update_layout(
    template = 'plotly_white',
    xaxis_title="Total Cases",
    xaxis_type='log',
    yaxis_title="Region",
    font=dict(size=14, color='#686868')
)
fig.show()

## Italy - China (Hubei province) comparison

Hubei province, where the virus initially started to spread, has approximately the same population of Italy (~60 million). A comparison between recovered, dead and total cases can be useful to assess the impact of the measures employed to limit the outbreak.

In [ ]:
data_world = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
data_world = data_world.drop(columns=['ObservationDate'])
data_world['Date'] = pd.to_datetime(data_world['Last Update']).dt.date
data_world['Actual cases']=data_world['Confirmed']-data_world['Deaths']-data_world['Recovered']

data_hubei=data_world[data_world['Province/State']=='Hubei']

In [ ]:
temp = data_hubei.groupby('Date')['Confirmed', 'Recovered', 'Deaths', 'Actual cases'].sum().reset_index()
# temp = temp.melt(id_vars="Data", value_vars=['Guariti', 'Deceduti', 'Attualmente Positivi'],
#                  var_name='Caso', value_name='Count')

fig = go.Figure(data=[
    go.Bar(name='Actual cases', x=temp['Date'], y=temp["Actual cases"], marker=go.bar.Marker(
    color=act)),
    go.Bar(name='Recovered', x=temp['Date'], y=temp["Recovered"], marker=go.bar.Marker(
    color=rec)),
    go.Bar(name='Deceased', x=temp['Date'], y=temp["Deaths"], marker=go.bar.Marker(
    color=dth))
])
# Change the bar mode
fig.update_layout(
    width=700,
    height=500,
    barmode='stack', 
    title='Total cases',
    font=dict(size=14, color='#686868'),
    xaxis_tickangle=-45,
    xaxis=dict(
        nticks=10
        ),
    yaxis=dict(
        title='Count'
        ))

fig.show()

In [ ]:
from plotly.subplots import make_subplots
temp = data_hubei.groupby('Date')['Deaths'].sum().reset_index()

fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Scatter(
    x=temp['Date'], y=temp['Deaths'], 
    name = 'Hubei deaths',
    mode='lines+markers',
    marker=dict(
        color='#4285f4'
    )
),row=1, col=1)

fig.add_trace(go.Scatter(
    x=temp['Date'], y=dth_ita['death'],
    name = 'Italy deaths (translated date)',
    mode='lines+markers',
        marker=dict(
        color='#ea4335'
    )
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=temp['Date'], y=temp['Deaths'], 
    mode='lines+markers',
    showlegend=False,
    marker=dict(
        color='#4285f4'
    )
),row=1, col=2)

fig.add_trace(go.Scatter(
    x=temp['Date'], y=dth_ita['death'],
    mode='lines+markers',
    showlegend=False,
    marker=dict(
        color='#ea4335'
    )
), row=1, col=2)

fig.update_layout(
    width=800,
    height=500,
    font=dict(size=14, color='#686868'),
    yaxis=dict(
        title='Deceased',
        range=[0.5, 4]
        ),
    yaxis_type='log',
    legend=dict(x=0.2, y=1.15),
    legend_orientation = 'h'
)

fig.update_xaxes(tickangle=-45)

fig.show()